In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# set the directory containing the images
dir_path = './'

# loop through each file in the directory
for file_name in os.listdir(dir_path):
    # check if the file is an image
    if file_name.endswith('.tif') or file_name.endswith('.jpg'):
        # get the file name without the extension
        name = os.path.splitext(file_name)[0]
        # use the file name to create a new directory
        output_dir = os.path.join(dir_path, name)
        os.makedirs(output_dir, exist_ok=True)
        # run the code to process the image
        # Load the image
        img = cv2.imread(os.path.join(dir_path, file_name))

        # Apply Otsu's thresholding
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

        # Apply morphological operations to remove noise and fill small gaps between cells
        kernel = np.ones((3,3), np.uint8)
        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

        # Perform distance transform to identify cell centers
        dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 3)
        ret, sure_fg = cv2.threshold(dist_transform, 0.1*dist_transform.max(), 255, 0)

        # Perform watershed segmentation to separate overlapping cells
        sure_fg = np.uint8(sure_fg)
        unknown = cv2.subtract(opening, sure_fg)
        ret, markers = cv2.connectedComponents(sure_fg)
        markers = markers + 1
        markers[unknown == 255] = 0
        markers = cv2.watershed(img, markers)

        # Draw contours around the cells and add additional features
        contours, hierarchy = cv2.findContours(markers.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
        features_list = []
        a = 0
        for i in range(len(contours)):
            # Find the bounding box around each cell
            x, y, w, h = cv2.boundingRect(contours[i])
            # Skip cells that are too small
            if w*h > 1000:
                 # Determine if the cell touches the image margin
                margin_threshold = 1  # Adjust this value if needed
                touches_margin = (x <= margin_threshold or y <= margin_threshold or x + w >= img.shape[1] - margin_threshold or y + h >= img.shape[0] - margin_threshold)

                # Draw the bounding box on the image
                if not touches_margin:
                    # Find the center point of the cell
                    cx = int(x + w/2)
                    cy = int(y + h/2)
                    # Create a mask for the cell
                    mask = np.zeros_like(img)
                    cv2.drawContours(mask, contours, i, (255, 255, 255), -1)
                    # Apply the mask to the original image to extract the cell region
                    cell = cv2.bitwise_and(img, mask)
                    # Crop the cell region
                    cell = cell[y:y+h, x:x+w]
                    # Check if the cell dimensions are less than or equal to 500x500
                    if cell.shape[0] > 500 or cell.shape[1] > 500:
                        continue
                    # Add padding to make the cell image the desired output size
                    cell_h, cell_w, _ = cell.shape
                    if cell_h > cell_w:
                        pad = (cell_h - cell_w) // 2
                        cell = cv2.copyMakeBorder(cell, 0, 0, pad, pad, cv2.BORDER_CONSTANT, value=(0, 0, 0))
                    else:
                        pad = (cell_w - cell_h) // 2
                        cell = cv2.copyMakeBorder(cell, pad, pad, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))

                    # Determine the desired output size
                    # why 500? Because the dimension of the image (max) is 500
                    output_size = (500, 500)

                    # Get the dimensions of the input image
                    input_size = cell.shape[:2]

                    # Compute the amount of padding needed to reach the desired output size
                    padding_width = output_size[0] - input_size[0]
                    padding_height = output_size[1] - input_size[1]

                    # Determine the amount of padding to add to each side of the input image
                    pad_top = padding_height // 2
                    pad_bottom = padding_height - pad_top
                    pad_left = padding_width // 2
                    pad_right = padding_width - pad_left

                    # Add the padding to the image
                    cell_img = cv2.copyMakeBorder(cell, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=(255, 255, 255))

                    
                    # Make the image background white
                    cell_img[cell_img == 0] = 255

                    # Save the cell image
                    cv2.imwrite(os.path.join(output_dir, f'{name}_{i+1}.png'), cell_img)
                    a = a + 1

        # Print the total number of cells detected for each image
        print(f'Total number of cells detected in {file_name}: {a}')


Total number of cells detected in MDA231 10k 01 D1.tif: 32
Total number of cells detected in MDA231 10k 01 D2.tif: 38
Total number of cells detected in MDA231 10k 01 D3.tif: 241
Total number of cells detected in MDA231 10k 02 D1.tif: 24
Total number of cells detected in MDA231 10k 02 D2.tif: 17
Total number of cells detected in MDA231 10k 02 D3.tif: 265
Total number of cells detected in MDA231 20k 01 D1.tif: 51
Total number of cells detected in MDA231 20k 01 D2.tif: 52
Total number of cells detected in MDA231 20k 01 D3.tif: 646
Total number of cells detected in MDA231 20k 02 D1.tif: 44
Total number of cells detected in MDA231 20k 02 D2.tif: 73
Total number of cells detected in MDA231 20k 02 D3.tif: 238
Total number of cells detected in MDA231 40k 01 D1.tif: 63
Total number of cells detected in MDA231 40k 01 D2.tif: 411
Total number of cells detected in MDA231 40k 01 D3.tif: 742
Total number of cells detected in MDA231 40k 02 D1.tif: 61
Total number of cells detected in MDA231 40k 02 D2